# AOC 2023

https://adventofcode.com/2023

In [238]:
from typing import List
from collections import Counter

## Utils

In [209]:
class Parser:
    @staticmethod
    def _read_file(filename, pp) -> List[str]:
        return [pp(l.strip()) for l in open(filename, "r")]

    @staticmethod
    def _read_test_input(string, pp) -> List[str]:
        return [pp(l.strip()) for l in string.split("\n") if len(l.strip()) > 0]
    
    @staticmethod
    def parse_input(sample_str, filename, pp=lambda x: x):
        return Parser._read_test_input(sample_str, pp), Parser._read_file(filename, pp)

## Day 1
    

### Part 1

Find the first and last digit.

In [211]:
sample_str = """
            1abc2
            pqr3stu8vwx
            a1b2c3d4e5f
            treb7uchet
            """

sample_input, file_input = Parser.parse_input(sample_str, "data/day1.txt")

In [212]:
def d1p1(test=False):
    input = sample_input if test else file_input
    
    first, last = None, None
    def find_digit(line):
        for c in line:
            if c.isdigit():
                return c
        return None

    results = []
    for line in input:
        first, last = find_digit(line), find_digit(line[::-1])
        results.append(int(first + last))
        
    return sum(results)

In [309]:
assert day1(True) == 142
day1()

54644

### Part 2

Since we know that digit names are of length 3, 4, or 5 - we can check if a length k digit name matches from a given index. Do this if its not already a digit.

In [311]:
s2d = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9"
}
s2d_rev = {k[::-1]: v for k, v in s2d.items()}

sample_str = """
    two1nine
    eightwothree
    abcone2threexyz
    xtwone3four
    4nineeightseven2
    zoneight234
    7pqrstsixteen
"""
sample_input, file_input = Parser.parse_input(sample_str, "data/day1.txt")

def valid_digitname(str, start, valid_s2d):
    # Is there a valid digitname from start index in the input string
    
    ends = [3, 4, 5]
    # Possible lengths for valid digit names
    ends = [3, 4, 5]
    for e in ends:
        candidate_name = str[start: start+e]
        if candidate_name in valid_s2d:
            return valid_s2d[candidate_name]
    return None

def find_digit(line, valid_s2d):
        for i in range(0, len(line), 1):
            c = line[i]
            if c.isdigit():
                return c
            elif (d := valid_digitname(line, i, valid_s2d)):
                return d
        return None

def d1p2(test=False):
    input = sample_input if test else file_input
    results = []
    for line in input:
        first = find_digit(line, s2d)
        last = find_digit(line[::-1], s2d_rev)
        results.append(int(first + last))
        
    return sum(results)

In [312]:
assert d1p2(True) == 281
d1p2()

53348

## Day 2

### Part 1

This seems like a form of CSP. We have some predicate (valid cubes), for each line - check which lines satisfy the predicate.

In [313]:
sample_str = """
    Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
    Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
    Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
    Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
    Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
"""

def parse_line(line):
    game, rest = line.split(":")
    game_id = int(game.replace("Game ", "").strip())
    
    games = []
    for g in rest.split(";"):
        counts = Counter()
        for hand in g.split(","):
            n, color = hand.strip().split(" ")
            counts[color.strip()] += int(n.strip())
            
        games.append(counts)
    return game_id, games
            
sample_input, file_input = Parser.parse_input(sample_str, "data/day2.txt", pp=parse_line)

In [314]:
total_cubes = {"red": 12, "green": 13, "blue": 14}
valid_hand = lambda picked: (picked["red"] <= total_cubes["red"] and
    picked["green"] <= total_cubes["green"] and 
    picked["blue"] <= total_cubes["blue"])

def d2p1(test=False):
    input = sample_input if test else file_input
    valid_ids = []
    for game_id, game in input:
        if all([valid_hand(hand) for hand in game]):
            valid_ids.append(game_id)

    return sum(valid_ids)

In [315]:
assert d2p1(True) == 8
d2p1()

2551

### Part 2

In [318]:
def update_min_cubes(candidate, best):
    for c in ["red", "green", "blue"]:
        best[c] = max(candidate[c], best[c])

def game_power(hand):
    return hand["red"] * hand["blue"] * hand["green"]

def d2p2(test=False):
    input = sample_input if test else file_input
    powers = []
    for game_id, game in input:
        min_cubes = {"red": 0, "blue": 0, "green": 0}
        for hand in game:
            update_min_cubes(hand, min_cubes)
        powers.append(game_power(min_cubes))
        
    return sum(powers)

In [319]:
assert d2p2(True) == 2286
d2p2()

62811

## Day 3